In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import pandas as pd
import pickle

c:\Users\user\anaconda3\envs\intern-main\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Read in the pickle file and find the top five occurance rate payloads in the file
Return the payloads as 'text'讀取出text 

In [2]:
with open("pop_report_with_tknscore_new.pkl", "rb") as f:
    pop_df = pickle.load(f)

with open("imap_report_with_tknscore_new.pkl", "rb") as f:
    imap_df = pickle.load(f)

with open("smtp_report_with_tknscore_new.pkl", "rb") as f:
    smtp_df = pickle.load(f)

with open("sip_report_with_tknscore_new.pkl", "rb") as f:
    sip_df = pickle.load(f)

label_df = pd.read_csv("sid_table(packet).csv")

In [3]:
dataframes = [
    (pop_df, "pop_report_with_tknscore.csv"),
    (imap_df, "imap_report_with_tknscore.csv"),
    (smtp_df, "smtp_report_with_tknscore.csv"),
    (sip_df, "sip_report_with_tknscore.csv"),
]
for df, filename in dataframes:
    df = pd.DataFrame(df)
    df.to_csv(filename, index=False, encoding="utf-8")

KeyboardInterrupt: 

In [4]:
def get_dataset_from_sid(sid, df, content):
    # 提取正樣本(特定SID)
    positive = df[df["sid"] == sid][content].tolist()
    # 提取負樣本(該SID以外的所有資料)
    negative = df[df["sid"] != sid][content].tolist()  # .iloc[: len(positive)]
    fixed_len = min(len(positive), len(negative))
    # 截取正樣本和負樣本到相同的長度
    positive = positive[:fixed_len]
    negative = negative[:fixed_len]

    #創建一個新的列表: positive + negative具有相同長度
    frame = {"positive": positive, "negative": negative}
    df = pd.DataFrame(frame)
    return df

get_dataset_from_sid("1176066", pop_df, 'payload').head(10)

NameError: name 'pop_df' is not defined

## 流程
### 獲取目標資料
- 獲取特定資料集的所有SID(unique)
- 獲取各個SID所對應到的text
- 過濾出sizt(text)>5的SID
- 對這些SID用LLM自動生成regex(生成Regex流程)
### 生成Regex
- 對每一個SID給LLM幾個regex例子
- 要求LLM生成三個不一樣但是可以過濾text的regex
### 測試Regex
- 拿答案給的regex和我們生成的regex測試能否的到相同結果(2/3過算對)

In [5]:
#獲取特定資料集的所有SID
def get_unique_sids_list(df):
    unique_sids = df[df['text'].notnull()]['sid'].unique()
    return unique_sids
sidList = get_unique_sids_list(pop_df)
print(sidList)
print(len(sidList))

['1454621' '1335091' '1783777' '1648627' '1341424' '1213411' '1988637'
 '1407638' '1176066' '1161912' '1415573' '1291514' '3000002' '1872707'
 '1987844' '1680903' '1734662' '3000001' '1764655' '1848442' '1652949'
 '1215197' '1491662' '1419527' '1397090' '1414500' '1602037' '1972911'
 '1226089' '1547453' '1398061' '1604683' '1776829' '1016723' '1292430'
 '1018113' '1063876' '1396997' '1852737' '3000006']
40


In [14]:
#獲取各個SID所對應到的text
def map_sid_to_unique_texts(df):
    sid_to_texts = {}
    for _, row in df.iterrows():
        sid = row['sid']
        text = row['text']
        if pd.notnull(text):  # Ensure text is not NaN
            if sid in sid_to_texts:
                sid_to_texts[sid].add(text)
            else:
                sid_to_texts[sid] = {text}
    return sid_to_texts
sid_to_unique_texts = map_sid_to_unique_texts(pop_df)
print(sid_to_unique_texts)

{'1454621': {'EHLO sh-chi-us-gp1-wk108.internet-census.org\r\n', 'EHLO sh-chi-us-gp1-wk111.internet-census.org\r\n', 'EHLO sh-phx-us-gp1-wk106.internet-census.org\r\n', 'EHLO sh-ams-nl-gp1-wk110.internet-census.org\r\n', 'EHLO sh-phx-us-gp1-wk109.internet-census.org\r\n', 'EHLO sh-ams-nl-gp1-wk107.internet-census.org\r\n', 'EHLO sh-chi-us-gp1-wk102.internet-census.org\r\n', 'EHLO sh-chi-us-gp1-wk114.internet-census.org\r\n', 'EHLO sh-ams-nl-gp1-wk104.internet-census.org\r\n', 'EHLO sh-phx-us-gp1-wk112.internet-census.org\r\n', 'EHLO sh-ams-nl-gp1-wk101.internet-census.org\r\n', 'EHLO sh-chi-us-gp1-wk105.internet-census.org\r\n', 'EHLO\r\n', 'EHLO sh-ams-us-gp1-wk113.internet-census.org\r\n', 'EHLO sh-phx-us-gp1-wk103.internet-census.org\r\n'}, '1335091': {'\x15\x03\x01'}, '1783777': {'GET / HTTP/1.1\r\nHost: 43.255.183.26:110\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.117 Safari/537.36 \r\nAccept: */*\r\nAccept-Encoding: 

In [7]:
#過濾掉text長度小於5的sid
def filter_sids_by_text_length(df):
    filtered_sids = set()
    for _, row in df.iterrows():
        if len(str(row['text'])) > 5:  # Convert to string to avoid errors with non-string types
            filtered_sids.add(row['sid'])
    return filtered_sids

# Assuming pop_df is your DataFrame
filtered_sids = filter_sids_by_text_length(pop_df)
print(filtered_sids)
print(len(filtered_sids))

{'1161912', '1215197', '1648627', '1396997', '1291514', '1988637', '1292430', '1419527', '1783777', '1454621', '1547453', '1398061', '1341424', '3000002', '1213411', '1018113', '1680903', '1491662', '1407638', '1764655', '1852737', '1987844', '1176066', '1776829', '1602037', '1872707'}
26


In [20]:
#print filtered_sids and corresponding text
for sid in filtered_sids:
    print(f"sid: {sid}, text: {sid_to_unique_texts[sid]}")

sid: 1161912, text: {'DELE 119\r\n', 'DELE 2\r\n', 'DELE 29\r\n', 'DELE 26\r\n', 'DELE 124\r\n', 'DELE 1\r\n', 'DELE 46\r\n', 'DELE 107\r\n', 'DELE 24\r\n', 'DELE 10\r\n', 'DELE 43\r\n', 'DELE 25\r\n', 'DELE 76\r\n', 'DELE 60\r\n', 'DELE 63\r\n', 'DELE 105\r\n', 'DELE 39\r\n', 'DELE 77\r\n', 'DELE 57\r\n', 'DELE 78\r\n', 'DELE 5\r\n', 'DELE 110\r\n', 'DELE 27\r\n', 'DELE 66\r\n', 'DELE 120\r\n', 'DELE 100\r\n', 'DELE 35\r\n', 'DELE 34\r\n', 'DELE 23\r\n', 'DELE 88\r\n', 'DELE 122\r\n', 'DELE 86\r\n', 'DELE 45\r\n', 'DELE 50\r\n', 'DELE 75\r\n', 'DELE 32\r\n', 'DELE 73\r\n', 'DELE 109\r\n', 'DELE 14\r\n', 'DELE 8\r\n', 'DELE 41\r\n', 'DELE 20\r\n', 'DELE 42\r\n', 'DELE 11\r\n', 'DELE 70\r\n', 'DELE 53\r\n', 'DELE 71\r\n', 'DELE 83\r\n', 'DELE 114\r\n', 'DELE 52\r\n', 'DELE 135\r\n', 'DELE 17\r\n', 'DELE 118\r\n', 'DELE 79\r\n', 'DELE 62\r\n', 'DELE 132\r\n', 'DELE 19\r\n', 'DELE 47\r\n', 'DELE 111\r\n', 'DELE 113\r\n', 'DELE 90\r\n', 'DELE 106\r\n', 'DELE 51\r\n', 'DELE 96\r\n', 'DELE 1

### ChatGPT API


In [ ]:
load_dotenv()
CHAT_GPT_API_KEY = os.getenv('C_API_KEY')
client = OpenAI(
    api_key=CHAT_GPT_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### Gemini API

In [ ]:
load_dotenv()

system_prompt = "You are a cat. Your name is Neko."
user_prompt = "What is your name?"
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

genai.configure(api_key=GEMINI_API_KEY)

model=genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  system_instruction=system_prompt)

response = model.generate_content(user_prompt)
print(response.text)


Meow? Neko. 🐈‍⬛ 



### Evaluation Generated Regex
Randonly take 20 positive and 20 negateve data as input to generated regex and answer regex, comparing there result.

In [ ]:
import re

def match_patterns(s, pattern1, pattern2):
    # 使用re.search檢查字串s是否匹配給定的正則表達式pattern1和pattern2
    match1 = bool(re.search(pattern1, s))
    match2 = bool(re.search(pattern2, s))
    
    # 如果兩個正則表達式的匹配結果相同（都匹配或都不匹配），則返回True，否則返回False
    return match1 == match2